Math087 - Mathematical Modeling
===============================
[Tufts University](http://www.tufts.edu) -- [Department of Math](http://math.tufts.edu)  
[George McNinch](http://gmcninch.math.tufts.edu) <george.mcninch@tufts.edu>  
*Fall 2020*

Course material (Week 7): Example: financial markets
--------------------------------------------------------

Financial market example
=========================

Consider the state of a financial market from week to week. 

- by a *bull market* we mean a week of generally rising prices. 
- by a *bear market* we mean a week of genreally declining prices.
- by a *recession* we mean a general slowdown of the economy.

Empirical observation shows for each of these three states what the probability of the state for the subsequent week, as follows:

|                  | *bull*   | *bear*   | *recession*| 
| :--------------- | -------: | -------: | ---------: |
|     followed by bull         | 0.90     | 0.15     | 0.25      |
|     followed by bear         | 0.075    | 0.80     | 0.25       |
|     followed by recession    | 0.025    | 0.05     | 0.50       |

In words, the first col indicates that if one has a bull market, then 90% of the time the next week is a bull market, 7.5% of the time the next week is a bear market, and 2.5% of the time the next week is in recession.

Probabilities
-------------

Let's number the weeks we are going to consider $k=0,1,2,...$.
We can represent the probability that week $k$ is a bull market, a bear market, or in recession using a vector in $\mathbb{R}^3$:

$$\mathbf{x}^{(k)} = \begin{bmatrix}
\text{bull market prob.} \\
\text{bear market prob.} \\
\text{recession prob.}
\end{bmatrix}
=\begin{bmatrix}
\alpha_k \\
\beta_k \\
\gamma_k
\end{bmatrix}$$

We'd like to describe the probabilities describing the next week -- i.e. the vector $\mathbf{x}^{(k+1)}$.

Observe that if we *knew for sure* that week $k$ was a bear market, then $\mathbf{x}^{(k)} = \mathbf{e}_1 = 
\begin{bmatrix} 
1 \\ 0 \\ 0 
\end{bmatrix}$.
But then we have more-or-less been told what $\mathbf{x}^{(k+1)}$ is -- it must be 
$$\mathbf{x}^{(k+1)} = \begin{bmatrix}
0.90\\
0.075 \\
0.025
\end{bmatrix}.$$

In the general case, we see if $\mathbf{x}^{(k)} = \begin{bmatrix}
\alpha_k \\
\beta_k \\
\gamma_k
\end{bmatrix}$, then
$$\mathbf{x}^{(k+1)} = \alpha_k \begin{bmatrix}
0.90\\
0.075 \\
0.025
\end{bmatrix} + 
\beta_k \begin{bmatrix}
0.15\\
0.8 \\
0.05
\end{bmatrix} +
\gamma_k \begin{bmatrix}
0.25\\
0.25 \\
0.5
\end{bmatrix} = A \cdot \begin{bmatrix} \alpha_k \\ \beta_k \\ \gamma_k \end{bmatrix} = A \cdot \mathbf{x}^{(k)}$$
where
$$A = \begin{bmatrix}
0.90 & 0.15 & 0.25 \\
0.075 & 0.80  & 0.25  \\
.025 & 0.05  & 0.50 
\end{bmatrix}$$

We want to enter $A$ as a matrix in ``python``, and use ``numpy`` functions 
to study the iterations.

Of course, with just three states this is fairly straightforward -- one just gets a $3 \times 3$ matrix. But with more variables, it might be nice to be more systematic. So I wanted to describe a way of using ``dictionaries`` in ``python`` to index the probabilities.

The idea is to make a list of the possible states; in this case

```
states = ["bull","bear","recess"]
```

and to use these states as the row/column index for the information in the resulting matrix.

So we just enter the matrix:

```
A = np.array([[0.90 , 0.15 , 0.25],
              [0.075, 0.80 , 0.25],
              [0.025, 0.05 , 0.50]])
```

and build the dictionary ``dict`` using the information in ``A``.

Note e.g. that if you wanted the entry in A corresponding to "bull then bear", you could do

``A[1,2]``

or

``A[states.index("bull"),states.index("bear")]``


In [ ]:
import numpy as np

A = np.array([[0.90 , 0.15 , 0.25],
              [0.075, 0.80 , 0.25],
              [0.025, 0.05 , 0.50]])

states = ["bull","bear","recess"]

A[states.index("bull"),states.index("bear")]

Of cours, writing

``A[states.index("bull"),states.index("bear")]``

is a bit clunky...

So let's create the dictionary I mentioned. A dictionary
is a collection of ``key/value`` pairs.

The *keys* to our dictionary will be pairs ("tuples")

``(a,b)`` where ``a,b`` are members of the list ``states``.

And the value associated to ``(a,b)`` will be the probability that state ``b`` follows state ``a``.

The dictionary will be named ``prob``, and we'll be able to the probability
that a bull market follows  a recession as follows:

``prob[("recess","bull")]``

To define ``dict``, we'll use:

```
{(a,b):A[states.index(a)][states.index(b)] for (a,b) in product(states,states)}
```

In [ ]:
from itertools import product

prob = {(a,b):A[states.index(a)][states.index(b)] for (a,b) in product(states,states)}

prob


In [ ]:
print("\n".join([f"week of {a:6} ---> week of {b:6}: {prob[(a,b)]}" for (a,b) in prob.keys()]))

In [ ]:
from graphviz import Digraph

fin = Digraph("financial")

## make the nodes
for a in states:
    fin.node(a)

## make and label the edges
for a,b in prob.keys():
    fin.edge(a,b,f"{prob[(a,b)]}")


fin

In [ ]:
def sbv(index,size):
    return np.array([1.0 if i == index-1 else 0.0 for i in range(size)])

state_vector = {"bull":sbv(1,3),
                "bear":sbv(2,3),
                "recess":sbv(3,3)}

def state(bull=0,bear=0,recess=0):
    if np.abs(1 - (bull + bear + recess)) < 1e-7:
        return sum([bull*state_vector["bull"],
                    bear*state_vector["bear"],
                    recess*state_vector["recess"]],
                    np.zeros(3))
    else:
        raise Exception("Probabilities must add to 1")
        
[A @ state_vector["bull"],
 A @ state(bull=1),
 A @ state(bull=.5,bear=.5)]

**Q1**
======

If there is a bear market in week $k$, what probabilities describe the market state 3 weeks later (i.e. in week $k+3$)?

In [ ]:
## we'll need to use the ``matrix_power`` function

from numpy.linalg import matrix_power


**Q2**
------

Suppose that in we knew that there was a 50% chance of a recession and a 50% chance of a bear market.
What probabilities describe the market state 5 weeks later?

**Q3**
=======

Compute A^50. What do you observe? What about A^100?

Explain what seems to happen to the vector $A^j \cdot \mathbf{x}^{(0)}$ for large $j$.


In [ ]:
matrix_power(A,50)

**Q4**
======
In the long run, what do you think the probability of a bull market is? Does this probability depend on the starting state of the market?

Bipartite Graphs & Matching
===========================

Bi-partite graphs
------------------

Let $G$ be an (undirected) graph with vertex set $V$ and edge set $E$.

Suppose that we can divide the vertices into two disjoint collections --
i.e. suppose that we can represent $V$ as a *disjoint union*:

$$V = U \cup W.$$


Recall that an edge $e$ of $G$ connects two vertices: $e = [x,y]$ for
$x,y \in V$.  We say that $G$ is a *bi-partite* graph if for each edge
$e = [x,y]$, we have $x \in U$ and $y \in W$ (or vice versa). In
words, every edge connects a vertex in $U$ and a vertex in $W$.

Thus, we do not allow edges to connect a pair of vertices in $U$ or a
pair of vertices in $W$.



In [ ]:
from graphviz import Graph

g1 = Graph()
for i in range(1,4):
    g1.node(f"A{i}")

for i in range(1,5):
    g1.node(f"B{i}")
    
g1.edge('A1','B2')
g1.edge('A1','B3')
g1.edge('A2','B1')
g1.edge('A2','B2')
g1.edge('A2','B3')
g1.edge('A3','B2')
g1.edge('A3','B5')
g1.edge('A3','B4')
    
g1


Matchings
---------

Let $G$ be a bi-partite graph, and suppose its vertex set $V$ is
written as a disjoint union $U \cup W$ as above.

A *matching* in $G$ is a set $M$ of edges in $E$ with the property that
no two edges in $M$ have a common vertex.

- A matching $M$ is *maximal* if $M \cup \{e\}$ is not a matching for
  any edge $e \not \in M$. 
  
  Thus $M$ is maximal if no larger set of edges is a matching.
  
- A matching $M$ has *maximum size* if there are no matchings $\tilde
  M$ in $G$ for which $|M| < |\tilde M|$ where $|X|$ denotes the
  *cardinality* of a set $X$ (i.e. $|X|$ denotes the number of
  elements in $X$, at least when $X$ is finite).
  
  Thus the matching $M$ has maximum size if there are no matchings
  with more edges.

- A matching is *perfect* if every vertex in $V$ is adjacent to one
  edge in the matching.


In [ ]:
import numpy as np
from scipy.optimize import linprog
from itertools import product

## the function find_matching defined below finds a maximal matching
## for a bi-partite graph

## the graph is given by 3 pieces of data: a set `U`, a set `W`, and a
## set `edge` of pairs (u,v) in the product U x V

## the matching is found by solving a linear program. Recall that we
## first associate a directed graph to the bi-partite graph, and solve
## the ``max-flow`` linear program for that directed graph.

def sbv(index,size):
    return np.array([1.0 if i == index else 0.0 for i in range(size)])

def to_vector(l,X):
    ## argument X is a list, and l is a list of elements from X.
    ## suppose that l=[x1,x2,...,xm] and suppose that
    ## n1,n2,...,nm are the indices of the xi in the list X.
    ## 
    ## this function returns the sum of the standard basis vectors
    ## sbv(ni,#X) for i =1,...,m
    ii = map(X.index,l)
    return sum([sbv(i,len(X)) for i in ii],np.zeros(len(X)))

def find_matching(U,W,edges):
    UW = list(product(U,W))

    edge_dict = {(u,w):True if (u,w) in edges else False for (u,w) in UW}
    
    def U_node(x):
        # compute the row-vector corresponding to the conservation law
        # for nodes in U the directed graph has an edge s -> u for
        # each u in U, and an edge w -> t for each w in W, as well as
        # the edges u->w coming from the original bi-partite graph.
        # Thus the rows of the constraint matrix for the linear program
        # have length #U + (#U)(#W) + #W
        return np.block([to_vector([x],U),
                         (-1)*to_vector([(x,w) for w in W if edge_dict[(x,w)]],UW),
                         np.zeros(len(W))])
                     

    def W_node(x):
        # compute the row-vector corresponding to the conservation law for a node in W
        # this row again has length #U + (#U)(#W) + #W.
        return np.block([np.zeros(len(U)),
                         (-1)*to_vector([(u,x) for u in U if edge_dict[(u,x)]],UW),
                         to_vector([x],W)])

    ## construct the equality constraint matrix from conservation laws
    A = np.array([U_node(x) for x in U] + [W_node(x) for x in W])

    ## construct the row for the objective function
    c = np.block([np.ones(len(U)), np.zeros(len(UW)), np.zeros(len(W))])

    lp_result = linprog((-1)*c,
                        A_eq=A,
                        b_eq=np.zeros(len(U) + len(W)),
                        bounds=(0,1),
                        method='revised simplex')

    def compare(r,targ=0,ee=1e-5):
        return True if np.abs(r-targ)<ee else False

    def extract(vec):
        pv = [vec[k + len(U)] for k in range(len(UW))]
        test = [True if compare(x,1) or compare(x,0) else False for x in pv]
        if all(test):
            match = [(u,w)  for (u,w) in UW if compare(pv[UW.index((u,w))],1)]
            return match
        else:
            raise Exception("linprog solution not of correct form.")

    match=extract(lp_result.x)
    
    if lp_result.success:
        return match
    else:
        raise Exception("linprog failed")
    

def display_matching(U,W,edges):
    UW = list(product(U,W))
    match = find_matching(U,W,edges)
    l = [f"length of matching: {len(match)}"] 
    s = [f"{u}  ---> {w}" for (u,w) in match]
    return "\n".join(l+s)

def graph(U,W,edges):
    g = Graph()
    for u in U:
        g.node(f"U{u}")

    for w in W:
        g.node(f"W{w}")

    for (u,w) in edges:
        g.edge(f"U{u}",f"W{w}")
        
    return g



In [ ]:
U =list(range(10))
W =list(range(10))

e = [(1, 0), (1, 5), (2, 3), (2, 5), (2, 9), (3, 0), (4, 1), (4, 5), (5, 5),
         (5, 6), (5, 9), (6, 8), (7, 0), (7, 7), (8, 4)]

print(display_matching(U,W,e))

graph(U,W,e)

In [ ]:
def edges(U,W,prob=1):
    return [(a,b) for (a,b) in product(U,W) if np.random.rand() < prob]

U=W=range(6)
e = edges(U,W,.55)


print(display_matching(U,W,e))
graph(U,W,e)

